# tsam - Basic Example

Example usage of the time series aggregation module (tsam).

This notebook demonstrates:
1. Basic k-means aggregation
2. Hierarchical aggregation with extreme periods
3. Advanced aggregation with segmentation

Import pandas and the relevant time series aggregation class

In [24]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd

import tsam
from tsam import ClusterConfig, ExtremeConfig, SegmentConfig

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Input data 

Read in time series from testdata.csv with pandas

In [25]:
raw = pd.read_csv("testdata.csv", index_col=0)

Show a slice of the dataset

In [26]:
raw.head()

,GHI,T,Wind,Load
2009-12-31 23:30:00,0,-2.1,7.1,375.478394
2010-01-01 00:30:00,0,-2.8,8.6,364.541326
2010-01-01 01:30:00,0,-3.3,9.7,357.416844
2010-01-01 02:30:00,0,-3.2,9.8,350.191306
2010-01-01 03:30:00,0,-3.2,9.4,345.161449


Show the shape of the raw input data: 4 types of timeseries (GHI, Temperature, Wind and Load) for every hour in a year

In [27]:
raw.shape

(8760, 4)

Plot the original temperature data as a heatmap

In [ ]:
# Use tsam's built-in heatmap plotting
tsam.plot.heatmap(raw, column="T", period_hours=24, title="Original Temperature")

Use the `aggregate()` function with k-means clustering for eight typical days.

In [29]:
result_kmeans = tsam.aggregate(
    raw,
    n_periods=8,
    period_hours=24,
    cluster=ClusterConfig(method="kmeans"),
)

Access the typical periods from the result object

In [30]:
typPeriods = result_kmeans.typical_periods
typPeriods.head()

GHI        Load          T      Wind
  TimeStep                                           
0 0         0.000000  367.490203  15.005882  1.761765
  1         0.000000  356.918182  14.797059  1.908824
  2         0.000000  351.420331  14.585294  1.808824
  3         0.000000  351.686077  14.400000  1.758824
  4         4.029412  355.997460  14.232353  1.747059

Show shape of typical periods: 4 types of timeseries for 8*24 hours

In [31]:
print(f"Shape: {typPeriods.shape}")
print(
    f"Periods: {result_kmeans.n_periods}, Timesteps per period: {result_kmeans.n_timesteps_per_period}"
)

Shape: (192, 4)
Periods: 8, Timesteps per period: 24


Save typical periods to .csv file

In [32]:
typPeriods.to_csv(os.path.join("results", "testperiods_kmeans.csv"))

Reconstruct the original time series based on the typical periods

In [33]:
predictedPeriods = result_kmeans.reconstruct()

Plot the repredicted data

In [ ]:
# K-means reconstructed temperature heatmap
tsam.plot.heatmap(
    predictedPeriods,
    column="T",
    period_hours=24,
    title="K-means Reconstructed Temperature",
)

As seen, they days with the minimal temperature are excluded. In case that they are required they can be added to the aggregation as follow.

### Hierarchical aggregation including extreme periods

Use hierarchical clustering with extreme period preservation. This ensures the day with the minimum temperature and maximum load are included.

In [35]:
result_hier = tsam.aggregate(
    raw,
    n_periods=8,
    period_hours=24,
    cluster=ClusterConfig(method="hierarchical"),
    extremes=ExtremeConfig(
        method="new_cluster",
        min_timesteps=["T"],  # Preserve day with minimum temperature
        max_timesteps=["Load"],  # Preserve day with maximum load
    ),
)

Create the typical periods

In [36]:
typPeriods = result_hier.typical_periods
typPeriods.head()

GHI        Load         T      Wind
  TimeStep                                     
0 0         0.0  403.253822 -0.654502  3.541068
  1         0.0  394.008077 -0.949049  4.485353
  2         0.0  389.631672 -1.047231  3.068926
  3         0.0  391.161914 -1.243596  2.832854
  4         0.0  396.952828 -1.439960  2.596783

The aggregation can also be evaluated by indicators

In [37]:
# View accuracy metrics
print(result_hier.accuracy)
print("\nRMSE per column:")
print(result_hier.accuracy.rmse)

AccuracyMetrics(
  rmse=0.1059 (mean),
  mae=0.0751 (mean),
  rmse_duration=0.0352 (mean)
)

RMSE per column:
GHI     0.093828
Load    0.099253
T       0.086290
Wind    0.144376
Name: RMSE, dtype: float64


Save typical periods to .csv file

In [38]:
typPeriods.to_csv(os.path.join("results", "testperiods_hierarchical.csv"))

Repredict the original time series based on the typical periods

In [39]:
predictedPeriodsWithEx = result_hier.reconstruct()

Plot repredicted data

In [ ]:
# Hierarchical with extremes reconstructed temperature heatmap
tsam.plot.heatmap(
    predictedPeriodsWithEx,
    column="T",
    period_hours=24,
    title="Hierarchical + Extremes Reconstructed Temperature",
)

Now also the days with the minimal temperature are integrated into the typical periods.

### Advanced aggregation method

Combining hierarchical clustering with segmentation (reduced temporal resolution) and distribution-preserving representation.

In [41]:
result_advanced = tsam.aggregate(
    raw,
    n_periods=24,
    period_hours=24,
    cluster=ClusterConfig(
        method="hierarchical",
        representation="distribution_minmax",
    ),
    segments=SegmentConfig(n_segments=8),
)

In [42]:
predictedPeriodsAdvanced = result_advanced.reconstruct()

In [ ]:
# Advanced method reconstructed temperature heatmap
tsam.plot.heatmap(
    predictedPeriodsAdvanced,
    column="T",
    period_hours=24,
    title="Advanced Method Reconstructed Temperature",
)

### Comparison of the aggregations 
It was shown for the temperature, but both times all four time series have been aggregated. Therefore, we compare here also the duration curves  of the electrical load for the original time series, the aggregation with k-mean, and the hierarchical aggregation including peak periods.

In [ ]:
# Duration curve comparison using built-in function
tsam.plot.compare(
    {
        "Original": raw,
        "8 typ days": predictedPeriods,
        "8 typ days + peak": predictedPeriodsWithEx,
        "24 typ days + 8 seg": predictedPeriodsAdvanced,
    },
    column="Load",
    plot_type="duration_curve",
)

Or as unsorted time series for an example week

In [ ]:
# Time slice comparison - Load
tsam.plot.compare(
    {
        "Original": raw,
        "8 typ days": predictedPeriods,
        "8 typ days + peak": predictedPeriodsWithEx,
        "24 typ days + 8 seg": predictedPeriodsAdvanced,
    },
    column="Load",
    plot_type="time_slice",
    start="20100210",
    end="20100218",
)

In [ ]:
# Time slice comparison - GHI
tsam.plot.compare(
    {
        "Original": raw,
        "8 typ days": predictedPeriods,
        "8 typ days + peak": predictedPeriodsWithEx,
        "24 typ days + 8 seg": predictedPeriodsAdvanced,
    },
    column="GHI",
    plot_type="time_slice",
    start="20100210",
    end="20100218",
    title="Time Slice Comparison - Solar Irradiance (Feb 10-18)",
)